In [1]:
import pandas as pd

df=pd.read_csv("spotify_millsongdata.csv")

In [2]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [3]:
df=df.sample(4000)

In [4]:
df.shape

(4000, 4)

In [5]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [6]:
df=df.drop('link',axis=1).reset_index(drop=True)

In [7]:
df.head(5)

,artist,song,text
0,Waterboys,Is She Conscious,Is she really wide awake \r\nIs she conscious...
1,UB40,Since I Met You Lady,Since I met you lady \r\nMy whole life has ch...
2,Patti Smith,Trespassers,Life is designed \r\nWith unfinished lines \...
3,Lady Gaga,Alejandro,"I know that we are young, \r\nAnd I know that..."
4,Noa,Uri,I wish I had a son \r\nA little boy \r\nBrig...


In [8]:
df.tail()

,artist,song,text
3995,Clash,Red Angel Dragnet,I come from a long way away \r\nAnd I know a ...
3996,Electric Light Orchestra,Dreaming Of 4000,Saviour of the night came slowly on the silent...
3997,Linda Ronstadt,Blue Bayou,I feel so bad I got a worried mind \r\nI'm so...
3998,Ramones,Any Way You Want It,"Any way you want it, you can call me any day ..."
3999,Waylon Jennings,I Fall In Love So Easily,You don't know what's good for you you won't l...


In [9]:
df['text']=df['text'].str.lower().replace(r'^\w\s',' ').replace(r'\n','regex=True')

In [10]:
df.head(5)

,artist,song,text
0,Waterboys,Is She Conscious,is she really wide awake \r\nis she conscious...
1,UB40,Since I Met You Lady,since i met you lady \r\nmy whole life has ch...
2,Patti Smith,Trespassers,life is designed \r\nwith unfinished lines \...
3,Lady Gaga,Alejandro,"i know that we are young, \r\nand i know that..."
4,Noa,Uri,i wish i had a son \r\na little boy \r\nbrig...


In [11]:
import nltk
print(nltk.__version__)


3.8.1


In [14]:
import nltk
print(nltk.data.path)

In [15]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ashut\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [16]:
import nltk
from nltk.stem.porter import PorterStemmer

In [17]:
stemmer=PorterStemmer()

In [18]:
def token(txt):
    token=nltk.word_tokenize(txt)
    a=[stemmer.stem(w)for w in token]
    return" ".join(a)

In [19]:
token("you are beautiful")

'you are beauti'

In [20]:
df['text'].apply(lambda x:token(x))

0       is she realli wide awak is she consciou of her...
1       sinc i met you ladi my whole life ha chang sin...
2       life is design with unfinish line that anoth s...
3       i know that we are young , and i know that you...
4       i wish i had a son a littl boy bright , with b...
                              ...                        
3995    i come from a long way away and i know a fine ...
3996    saviour of the night came slowli on the silent...
3997    i feel so bad i got a worri mind i 'm so lones...
3998    ani way you want it , you can call me ani day ...
3999    you do n't know what 's good for you you wo n'...
Name: text, Length: 4000, dtype: object

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
tfid = TfidfVectorizer(analyzer='word',stop_words='english')

In [23]:
matrix=tfid.fit_transform(df['text'])

In [24]:
similer = cosine_similarity(matrix)

In [26]:
def recommender(somg_name):
    idx=df[df['song']==somg_name].index[0]
    distance=sorted(list(enumerate(similer[idx])),reverse=True,key=lambda x:x[1])
    song=[]
    for s_id in distance[1:5]:
        song.append(df.iloc[s_id[0]].song)
        return song

In [29]:
# recommender("Piece Of Me")


def recommender(song_name):
    # Check if the song name is present in the DataFrame
    if song_name not in df['song'].values:
        return "Song not found in the dataset."

    # Get the index of the song
    idx = df[df['song'] == song_name].index[0]

    # Assuming `similer` is your cosine similarity matrix
    distance = sorted(list(enumerate(similer[idx])), reverse=True, key=lambda x: x[1])
    recommendations = []

    # Get recommendations
    for i in range(1, 6):  # Getting top 5 recommendations excluding the song itself
        rec_song_idx = distance[i][0]
        rec_song_name = df.iloc[rec_song_idx]['song']
        recommendations.append(rec_song_name)

    return recommendations

# Example usage
print(recommender("Piece Of Me"))


Song not found in the dataset.


In [55]:
import pickle 

In [56]:
pickle.dump(similer,open("similarity","wb"))

In [57]:
pickle.dump(df,open("df","wb"))